In [14]:
from src.guitar.Guitar import Guitar
from src.hand.LeftFinger import LeftFinger
from src.hand.LeftHand import LeftHand
from src.guitar.GuitarString import createGuitarStrings
from src.midi.midiToNote import midiToGuitarNotes, processedNotes, get_tempo_changes,calculate_frame
from src.utils.utils import convertNotesToChord, convertChordTofingerPositions
from src.HandPoseRecorder import HandPoseRecorder,HandPoseRecordPool
from src.guitar.MusicNote import MusicNote
import itertools
import json

## Import Midi file 导入Midi文件
Import midi file and convert it to a list of dict which including notes and beat. 导入midi文件，并将其转换为一个列表，这个列表里的元素是包含音符和节拍信息的字典。

In [15]:
avatar = 'rem'
midiFilePath = "asset/midi/MARIO-1-1.mid"
filename = midiFilePath.split("/")[-1].split(".")[0]
left_hand_recorder_file = f"output/{filename}_lefthand_recorder.json"
left_hand_animation_file = f"output/{avatar}_{filename}_lefthand_animation.json"
right_hand_recorder_file = f"output/{filename}_righthand_recorder.json"
right_hand_animation_file = f"output/{avatar}_{filename}_righthand_animation.json"
FPS = 30

tempo_changes, ticks_per_beat = get_tempo_changes(midiFilePath)

notes_map = midiToGuitarNotes(midiFilePath, useChannel=1)

MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=333333, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('track_name', name='è¶\x85çº§ç\x8e\x9bä¸½', time=0),
    MetaMessage('set_tempo', tempo=400000, time=65280),
    MetaMessage('end_of_track', time=0)]),
  MidiTrack([
    Message('note_on', channel=0, note=64, velocity=89, time=0),
    Message('note_on', channel=0, note=60, velocity=89, time=0),
    Message('note_on', channel=0, note=54, velocity=89, time=0),
    Message('note_on', channel=0, note=50, velocity=89, time=0),
    MetaMessage('track_name', name='é\x9f³è½¨ 1', time=0),
    Message('program_change', channel=0, program=25, time=0),
    MetaMessage('key_signature', key='C', time=0),
    Message('control_change', channel=0, control=101, value=0, time=0),
    Message('control_change', channel=0, control=100, value=0, time=0),
    Mes

In [16]:
print(f'全曲的速度变化是:\n{tempo_changes}\n')
print(f'全曲的每拍tick数是:\n{ticks_per_beat}\n')
print(f'所有音符是:\n{notes_map}\n')
total_tick = notes_map[-1]['real_tick']
total_frame = calculate_frame(tempo_changes, ticks_per_beat, FPS,total_tick)
total_time = total_frame/FPS
print(f'如果以{FPS}的fps做成动画，一共是{total_tick} ticks,{total_frame}帧,约{total_time}秒')

全曲的速度变化是:
[(0, 333333, 0), (0, 400000, 65280)]

全曲的每拍tick数是:
480

所有音符是:
[{'notes': [50, 54, 60, 64], 'real_tick': 0}, {'notes': [50, 54, 60, 64], 'real_tick': 480}, {'notes': [55, 59, 64], 'real_tick': 720}, {'notes': [50, 54, 60, 64], 'real_tick': 960}, {'notes': [50, 54], 'real_tick': 1440}, {'notes': [50, 60, 64], 'real_tick': 1680}, {'notes': [59, 64], 'real_tick': 1920}, {'notes': [55, 59, 64, 67], 'real_tick': 2400}, {'notes': [48, 55, 60], 'real_tick': 4080}, {'notes': [55, 59], 'real_tick': 4320}, {'notes': [64], 'real_tick': 4560}, {'notes': [52, 55], 'real_tick': 4800}, {'notes': [50, 55], 'real_tick': 5040}, {'notes': [60], 'real_tick': 5280}, {'notes': [48, 52], 'real_tick': 5520}, {'notes': [45, 50], 'real_tick': 5760}, {'notes': [41], 'real_tick': 6000}, {'notes': [45], 'real_tick': 6480}, {'notes': [43, 59], 'real_tick': 6720}, {'notes': [50], 'real_tick': 6960}, {'notes': [58], 'real_tick': 7200}, {'notes': [41, 57], 'real_tick': 7440}, {'notes': [52, 55], 'real_tick':

## init guitar and handPoseRecordPool 初始化吉他和handPoseRecordPool
init guitar with a list of string. here we use a standard 6-string guitar. 用一个字符串列表来初始化吉他。这里我们使用标准的6弦吉他。
init the start hand pose. here we put a hand at barrel 0 and put fingers on string 3, which is G string. 初始化起始手型。在这里，我们将设定初始手型位置为第0品，并将手指放在第3弦上，也就是G弦上。

handPoseRecordPool can init with param `size`, which is the size of the pool. Bigger size will cost more memory and more time but can calculate more possibilities. handPoseRecordPool可以通过参数`size`初始化，即池的大小。越大就会消耗更多的内存和时间，但可以计算更多可能性。

In [17]:
# 设定各弦音高
guitar_string_list = createGuitarStrings(["e", "b", "G", "D", "A", "E1"])
# 初始化吉它
guitar = Guitar(guitar_string_list)
# 设定各手指状态
leftFingers = [
    LeftFinger(1, guitar_string_list[1], 1),
    LeftFinger(2, guitar_string_list[2], 2),
    LeftFinger(3, guitar_string_list[3], 3),
    LeftFinger(4, guitar_string_list[4], 4)
]
# 初始化左手
initLeftHand = LeftHand(leftFingers)
# 初始化第一个记录器
handPoseRecord = HandPoseRecorder()
handPoseRecord.addHandPose(initLeftHand, 0,0)
# 初始化记录池
handPoseRecordPool = HandPoseRecordPool(100)
handPoseRecordPool.insert_new_hand_pose_recorder(handPoseRecord,0)

current_recoreder_num = 0
previous_recoreder_num = 0

## Calculate the possibility of hand pose 计算手型的可能性
- For each notes, we calculate all possibile chords, which includes position infomations of notes on guitar. 对于每个音符，我们计算所有可能的和弦，包括音符在吉他上的位置信息。
- For each chords, we calculate all possibile hand poses, which includes the position of fingers on guitar.
- For each hand poses, we generate a LeftHand object if it is possible to play. 对于每个和弦，我们计算所有可能的手型，包括手指在吉他上的位置。
- For each LeftHand object, we calculate entropy, the cost of switch hand pose. Then we append it to the handPoseRecord. 对于每个手型，如果可以演奏，我们就生成一个LeftHand对象。
- For each handPoseRecord, we put it in the handPoseRecordPool if its entropy is lower than the record in the pool. 对于每个LeftHand对象，我们计算熵，即切换手型的成本。然后将其添加到handPoseRecord中。
- After all notes are calculated, we get the best handPoseRecord from the pool. Then print the result. 计算完所有音符后，我们从池中获取最佳handPoseRecord，然后打印结果


In [18]:
def generateLeftHandRecoder(guitarNote, current_recoreder_num:int, previous_recoreder_num:int):
    notes = guitarNote["notes"]
    real_tick = guitarNote["real_tick"]
    notes = processedNotes(notes)
    
    # calculate all possible chords, including the position information of notes on the guitar. 计算所有可能的和弦,包含音符在吉它上的位置信息。
    chords = convertNotesToChord(notes, guitar)
    
    # init current record list. 记录池先更新初始化当前记录列表。
    handPoseRecordPool.readyForRecord()
    fingerPositionsList = []
    handPoseRecordCount = 0

    # calculate all possible fingerings, including the position information of fingers on the guitar. 计算所有可能的按法，包含手指在吉它上的位置信息。
    for chord in chords:
        possibleFingerPositions = convertChordTofingerPositions(chord) 
        if len(possibleFingerPositions) == 0:
            continue       
        fingerPositionsList += possibleFingerPositions

    if len(fingerPositionsList) == 0:
        print(f"当前时间是{real_tick}，没有找到合适的按法。这是所有的chords：{chords}。")


    for handPoseRecord, fingerPositions in itertools.product(handPoseRecordPool.preHandPoseRecordPool, fingerPositionsList):
        oldhand = handPoseRecord.currentHandPose()

        # Iterate through the list of fingerings, generate a new LeftHand object based on the fingering. 遍历按法列表，根据按法生成新的LeftHand对象。
        all_fingers, entropy = oldhand.generateNextHands(
            guitar, fingerPositions, notes)

        if all_fingers is not None:
            new_entropy = handPoseRecord.currentEntropy + entropy
            insert_index = handPoseRecordPool.check_insert_index(new_entropy)
            # 当新手型符合插入记录器条件时
            if insert_index != -1:
                newHandPoseRecord = HandPoseRecorder()
                new_hand = LeftHand(all_fingers)
                
                newHandPoseRecord.handPoseList = handPoseRecord.handPoseList +[new_hand]
                newHandPoseRecord.currentEntropy = handPoseRecord.currentEntropy + entropy
                newHandPoseRecord.entropys = handPoseRecord.entropys + [new_entropy]
                newHandPoseRecord.real_ticks = handPoseRecord.real_ticks + [real_tick]   
                             
                handPoseRecordPool.insert_new_hand_pose_recorder(
                    newHandPoseRecord,insert_index)
                handPoseRecordCount += 1
    
    previous_recoreder_num = current_recoreder_num
    current_recoreder_num = len(handPoseRecordPool.curHandPoseRecordPool)

    if current_recoreder_num < previous_recoreder_num:
        print(f"当前record数量是{current_recoreder_num}，上一次record数量是{previous_recoreder_num}，这一轮操作一共append了{handPoseRecordCount}个record。")
        print(f"此时的real_tick是{real_tick},此时的notes是：{notes},对应的音符是：")        
        for note in notes:
            musice_note = MusicNote(note)
            print(musice_note.key)
    
    return current_recoreder_num, previous_recoreder_num

In [19]:
from tqdm import tqdm

total_steps = len(notes_map)
current_recoreder_num = 0
previous_recoreder_num = current_recoreder_num


def update_recorder_pool(notes_map, current_recoreder_num, previous_recoreder_num):
    with tqdm(total=total_steps, desc="Processing", ncols=100, unit="step") as progress:
        for i in range(0,total_steps):
            guitarNote = notes_map[i]
            current_recoreder_num, previous_recoreder_num = generateLeftHandRecoder(
                guitarNote, current_recoreder_num, previous_recoreder_num)
            progress.update(1)

update_recorder_pool(notes_map, current_recoreder_num, previous_recoreder_num)


# after all iterations, read the best solution in the recorder pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = handPoseRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output(True)
bestHandPoseRecord.save(left_hand_recorder_file)
print(f"总音符数应该为{total_steps}")
print(f"实际输出音符数为{len(bestHandPoseRecord.handPoseList)}")

Processing: 100%|███████████████████████████████████████████████| 238/238 [00:11<00:00, 20.52step/s]

最小消耗熵为：1077.7874721907272
Entropy:  12.770686459522558
real_tick:  0
4 | 4 string |  5 fret |   Partial_barre_2_strings
3 | 3 string |  4 fret |   Pressed
-1 | 0 string |  0 fret |   Open
1 | 2 string |  2 fret |   Open
2 | 2 string |  3 fret |   Open
0
0 -- -- -- -- -- -- -- --0
| -- -- -- -- -- -- -- --1
| -- 01 02 -- -- -- -- --2
| -- -- -- -3 -- -- -- --3
| -- -- -- -- -4 -- -- --4
| -- -- -- -- -- -- -- --5
把位：2
-------------------------------
Entropy:  12.820686459522559
real_tick:  480
4 | 4 string |  5 fret |   Partial_barre_2_strings
3 | 3 string |  4 fret |   Pressed
-1 | 0 string |  0 fret |   Open
1 | 2 string |  2 fret |   Open
2 | 2 string |  3 fret |   Open
0
0 -- -- -- -- -- -- -- --0
| -- -- -- -- -- -- -- --1
| -- 01 02 -- -- -- -- --2
| -- -- -- -3 -- -- -- --3
| -- -- -- -- -4 -- -- --4
| -- -- -- -- -- -- -- --5
把位：2
-------------------------------
Entropy:  14.570686459522559
real_tick:  720
4 | 3 string |  5 fret |   Pressed
3 | 2 string |  4 fret |   Pressed
-1 

In [20]:
# 给reacoder中添加frame信息

with open(left_hand_recorder_file, 'r') as f:
    left_hand_recorder = json.load(f)
    
    for item in left_hand_recorder:
        real_tick = item['real_tick']
        frame = calculate_frame(tempo_changes, ticks_per_beat, 24, real_tick)
        item['frame'] = frame

with open(left_hand_recorder_file, 'w') as f:
    json.dump(left_hand_recorder, f, indent=4)


## Initialize Animation Values 初始化动画所需数值

In Blender, by determining the positions of the following key points: IK pivot point, Y-axis rotation value, we can determine the entire left hand's posture. These key points are:
在blender中，通过确定以下几个关键点的位置，ik轴点，y轴旋转值，就可以确定整个左手的状态，这几个关键点分别是：
- Palm 手掌
- Thumb 大拇指
- Index finger 食指
- Middle finger 中指
- Ring finger 无名指
- Pinky finger 小指

There are four extreme positions of the hand shape, which are:
而手型有四个极端位置，分别是：
- Open string position on the 1st string 1弦空品
- Open string position on the 6th string 6弦空品
- 12th fret position on the 1st string 1弦12品
- 12th fret position on the 6th string 6弦12品

For each extreme position, we can adopt three methods to press the strings:
而每个极端位置，我们都可以采用三种方式来按弦，分别是：
- Index finger and middle finger on different frets, with the whole palm parallel to the neck 食指和中指在不同品上，整个手掌平行于琴颈
- Index finger and middle finger on the same fret, with the index finger pointing outward and the middle finger pointing inward, requiring the palm to rotate at a certain angle 食指与中指同品，而食指向外，中指向内，此时手掌要旋转一定角度
- Index finger and middle finger on the same fret, with the index finger pointing inward and the middle finger pointing outward, requiring the palm to rotate at a certain angle 食指与中指同品，而食指向内，中指向外，此时手掌要旋转一定角度

In Blender, we record the values of all key points for these four extreme positions in three different ways, totaling 12 hand shapes. 在blender中，我们记录下来以上四个极端位置的三种不同方式下，所有关键点的值，共计12种手型。

Through these 12 hand shapes, we can obtain the hand shape at any position through interpolation. 通过这12种手型，我们可以通过插值的方式，得到任意位置的手型。

In [21]:
from src.animate.animate import leftHand2Animation
# 生成动画文件

leftHand2Animation(avatar,left_hand_recorder_file, left_hand_animation_file, tempo_changes, ticks_per_beat, FPS)

## Calculate the right-hand movements 计算右手的动作

Similar to the principle of calculating the left hand, our left hand starts from an initial hand shape. Then, each time a string needs to be strummed, we calculate the possible next strumming hand shapes and the entropy consumed by each hand shape. These hand shapes are recorded in a rightHandRecorderPool. Ultimately, through repeated iteration, the strumming sequence that consumes the least entropy is selected.

和计算左手的原理类似，我们的左手从一个初始手型开始，然后每一次需要拨弦时，就计算下一次可能有哪些拨弦手型，并且计算每个手型所消耗的熵，把这些手型都记录在一个rightHandRecorderPool里，最终反复迭代，选出消耗熵最小的拨弦序列。

In [22]:
from src.hand.RightHand import RightHand
from src.HandPoseRecorder import RightHandRecorder,HandPoseRecordPool
initRightHand = RightHand(usedFingers=[],rightFingerPositions=[5,2,1,0])

initRightHandRecorder = RightHandRecorder()
initRightHandRecorder.addHandPose(initRightHand, 0,0)

rightHandRecordPool = HandPoseRecordPool(100)
rightHandRecordPool.insert_new_hand_pose_recorder(initRightHandRecorder,0)

In [23]:
def generateRightHandRecoder(item, rightHandRecordPool, current_recoreder_num, previous_recoreder_num):
    import itertools
    from src.hand.RightHand import generatePossibleRightHands
    
    real_tick = item["real_tick"]
    leftHand = item["leftHand"]
    usedStrings = []

    for finger in leftHand:
        if finger["fingerIndex"] == -1 or 5 > finger["fingerInfo"]["press"] > 0:
            usedStrings.append(finger["fingerInfo"]["stringIndex"])
            
    if usedStrings == []:
        return
    rightHandRecordPool.readyForRecord()

    allFingers = ["p","p","i","m","a"] # 这个重复p的写法是确保p指可能弹两根弦
    allstrings = [0,1,2,3,4,5]

    possibleRightHands = generatePossibleRightHands(
        usedStrings, allFingers, allstrings) 
    
    if len(possibleRightHands)==0:
        print(f"当前要拨动的弦是{usedStrings}，没有找到合适的右手拨法。")
    
    for rightHand,handRecorder in itertools.product(possibleRightHands,rightHandRecordPool.preHandPoseRecordPool): 
        lastHand = handRecorder.currentHandPose()
        entropy = lastHand.caculateDiff(rightHand)
        new_entropy = handRecorder.currentEntropy + entropy
        insert_index = rightHandRecordPool.check_insert_index(
            new_entropy)
        if insert_index != -1:
            newRecorder = RightHandRecorder()
            
            newRecorder.handPoseList = handRecorder.handPoseList + [rightHand]
            newRecorder.currentEntropy = handRecorder.currentEntropy + entropy
            newRecorder.entropys = handRecorder.entropys + [new_entropy]
            newRecorder.real_ticks = handRecorder.real_ticks + [real_tick]
            
            rightHandRecordPool.insert_new_hand_pose_recorder(
                newRecorder,insert_index)
    
    previous_recoreder_num = current_recoreder_num
    current_recoreder_num = len(
        rightHandRecordPool.curHandPoseRecordPool)

    if current_recoreder_num < previous_recoreder_num:
        print(f"当前record数量是{current_recoreder_num}，上一次record数量是{previous_recoreder_num}")

In [24]:
import json
from tqdm import tqdm

def update_right_hand_recorder_pool(left_hand_recorder_file, rightHandRecordPool, current_recoreder_num, previous_recoreder_num):
    with open(left_hand_recorder_file, "r") as f:
        data = json.load(f)
        total_steps = len(data)
        current_recoreder_num = 0
        previous_recoreder_num = current_recoreder_num
        
        with tqdm(total=total_steps, desc="Processing", ncols=100, unit="step") as progress:    
            for i in range(total_steps):
                item = data[i]
                generateRightHandRecoder(
                    item, rightHandRecordPool, current_recoreder_num, previous_recoreder_num)
                progress.update(1)


update_right_hand_recorder_pool(
    left_hand_recorder_file, rightHandRecordPool, current_recoreder_num, previous_recoreder_num)

# after all iterations, read the best solution in the record pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = rightHandRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output()
bestHandPoseRecord.save(right_hand_recorder_file)

Processing:   0%|                                                         | 0/238 [00:00<?, ?step/s]

Processing: 100%|███████████████████████████████████████████████| 238/238 [00:13<00:00, 18.16step/s]


最小消耗熵为：214
Entropy:  214
Entropy:  1
real_tick:  0
RightHand:  ['i', 'a'] [5, 3, 1, 0]
Entropy:  6
real_tick:  480
RightHand:  ['p', 'a'] [4, 2, 1, 0]
Entropy:  9
real_tick:  720
RightHand:  ['i', 'a'] [4, 2, 1, 0]
Entropy:  15
real_tick:  960
RightHand:  ['p', 'i', 'a'] [4, 2, 1, 0]
Entropy:  18
real_tick:  1440
RightHand:  ['p'] [4, 2, 1, 0]
Entropy:  19
real_tick:  1680
RightHand:  ['i', 'a'] [5, 2, 1, 0]
Entropy:  22
real_tick:  1920
RightHand:  ['i', 'm'] [5, 2, 1, 0]
Entropy:  29
real_tick:  2400
RightHand:  ['i', 'm'] [4, 2, 1, 0]
Entropy:  29
real_tick:  4080
RightHand:  ['p'] [4, 2, 1, 0]
Entropy:  29
real_tick:  4320
RightHand:  ['m'] [4, 2, 1, 0]
Entropy:  32
real_tick:  4560
RightHand:  ['m'] [4, 2, 1, 0]
Entropy:  32
real_tick:  4800
RightHand:  ['p'] [4, 2, 1, 0]
Entropy:  33
real_tick:  5040
RightHand:  ['i'] [5, 2, 1, 0]
Entropy:  35
real_tick:  5280
RightHand:  ['m'] [5, 3, 2, 0]
Entropy:  35
real_tick:  5520
RightHand:  [] [5, 3, 2, 0]
Entropy:  35
real_tick:  5760
Ri

In [25]:
with open(right_hand_recorder_file, 'r') as f:
    right_hand_recorder = json.load(f)    

    for item in right_hand_recorder:
        real_tick = item['real_tick']
        frame = calculate_frame(tempo_changes, ticks_per_beat, FPS, real_tick)
        item['frame'] = frame

with open(right_hand_recorder_file, 'w') as f:
    json.dump(right_hand_recorder, f, indent=4)

In [26]:
from src.animate.animate import rightHand2Animation

rightHand2Animation(avatar,right_hand_recorder_file,
                    right_hand_animation_file, tempo_changes, ticks_per_beat, FPS)